In [1]:
import IPython
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Create UI

A0_slider = widgets.FloatLogSlider(value=10, min=0, max=2.7, step=0.05, description='A0:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')
A1_checkbox = widgets.Checkbox(value=True, layout=Layout(flex='1 1 0%', width='auto', object_position="right center"))
A1_slider = widgets.FloatLogSlider(value=10, min=0, max=2.7, step=0.05, description='A1:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')

alpha0_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='α0:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')
alpha1_checkbox = widgets.Checkbox(value=True, layout=Layout(flex='1 1 0%', width='auto', object_position="right center"))
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='α1:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')

n0_slider = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='n0:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')
n1_checkbox = widgets.Checkbox(value=True, layout=Layout(flex='1 1 0%', width='auto', object_position="right center"))
n1_slider = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='n1:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')

delta0_slider = widgets.FloatSlider(value=0.25, min=0, max=1, step=0.01, description='δ0:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')
delta1_checkbox = widgets.Checkbox(value=True, layout=Layout(flex='1 1 0%', width='auto', object_position="right center"))
delta1_slider = widgets.FloatSlider(value=0.25, min=0, max=1, step=0.01, description='δ1:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')

s0_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='s0:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')
s1_checkbox = widgets.Checkbox(value=True, layout=Layout(flex='1 1 0%', width='auto', object_position="right center"))
s1_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='s1:', layout=Layout(flex='10 1 0%', width='auto'), disabled=False, continuous_update=False, readout=True, readout_format='.2f')

draw_btn = widgets.Button(
    description='Draw',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Draw',
    icon='drafting-compass'
)



row_layout = Layout(display='flex', flex_flow='row', align_items='stretch',width='100%')
A_row = Box(children=[A0_slider, A1_checkbox, A1_slider], layout=row_layout)
alpha_row = Box(children=[alpha0_slider, alpha1_checkbox, alpha1_slider], layout=row_layout)
n_row = Box(children=[n0_slider, n1_checkbox, n1_slider], layout=row_layout)
delta_row = Box(children=[delta0_slider, delta1_checkbox, delta1_slider], layout=row_layout)
s_row = Box(children=[s0_slider, s1_checkbox, s1_slider], layout=row_layout)
button_row = Box([draw_btn], layout=row_layout)

opt_box = VBox([A_row, alpha_row, n_row, delta_row, s_row, button_row])

display(opt_box)

# Function for Drawing
def draw(b):
    IPython.display.clear_output() #Hacky way of clearing output
    display(opt_box)

    # Options
    samples = 500 # Number of samples for each curve
    x_range_multiplier = 2 # How many times largest k* should x axis go to
    
    # Get all our first state model params
    first_state_vals = [A0_slider.value, alpha0_slider.value, n0_slider.value, delta0_slider.value, s0_slider.value];
    # Get all second state model params, setting to slider value if checkbox is ticked
    second_state_vals = [] #acum
    for i, checkbox, val in ((0, A1_checkbox.value, A1_slider.value), (1, alpha1_checkbox.value, alpha1_slider.value), (2, n1_checkbox.value, n1_slider.value), (3, delta1_checkbox.value, delta1_slider.value), (4, s1_checkbox.value, s1_slider.value)):
        if checkbox:
            second_state_vals.append(val)
        else:
            second_state_vals.append(first_state_vals[i])
    A0, alpha0, n0, delta0, s0 = first_state_vals
    A1, alpha1, n1, delta1, s1 = second_state_vals
    
    
    # We first want to find our steady_state variables, this will let us decide appropriate x ranges
    y_star0, k_star0, sy_star0 = steady_state_vars(A0, alpha0, n0, delta0, s0)
    y_star1, k_star1, sy_star1 = steady_state_vars(A1, alpha1, n1, delta1, s1)
    max_kstar = max(k_star0, k_star1)
    max_ystar = max(y_star0, y_star1)
    max_systar = max(sy_star0, sy_star1)
    
    
    # Generate array of samples to use as x axis
    x_vals = np.linspace(0, (x_range_multiplier*max_kstar), num=samples)

    # Close existing figures and Creates 1 fig and 3 axes 
    fig, (ax1, ax2, ax3) = plt.subplots(3, dpi=100, figsize=(6,18))
    ax1.set_ylim(0, max_systar*1.5)
    
    # ax1 for investment model
    ax1.title.set_text('Basic Solow Steady-State')
    
    # Get and plot on ax1 investment per person data
    investment_pc0 = investment_pc(x_vals, A0, alpha0, s0)
    pc0 = ax1.plot(x_vals, investment_pc0, label=r'$sy_0$', color="red")
    
    # If (s0, A0, alpha0) and (s1, A1, alpha1) differ, draw a second sy line
    if (s0, A0, alpha0) != (s1, A1, alpha1):
        investment_pc1 = investment_pc(x_vals, A1, alpha1, s1)
        ax1.plot(x_vals, investment_pc1, label=r'$sy_1$', color="red", linestyle="dashed")
    
    
    # Get and Plot on ax1 k(n+delta) data
    k_n_plus_delta0 = k_n_plus_delta(x_vals, n0, delta0)
    ax1.plot(x_vals, k_n_plus_delta0, label= r'$k(n+\delta)_0$', color="blue")
    
    # If ( n0, delta0) and ( n0, delta0) differ, draw a second sy line
    if (n0, delta0) != (n1, delta1):
        k_n_plus_delta1 = k_n_plus_delta(x_vals, n1, delta1)
        ax1.plot(x_vals, k_n_plus_delta1, label= r'$k(n+\delta)_1$', color="blue", linestyle="dashed")
    
    # Annotate Steady States
    ax1.scatter([k_star0,k_star1],[sy_star0,sy_star1])
    ax1.annotate(r'$k*_0:$' + str(np.round(k_star0,2)) + "\n" + r'$sy*_0:$' + str(np.round(sy_star0,2)), (k_star0, sy_star0), textcoords="offset points", xytext=(5,-25))
    if (k_star0, sy_star0) != (k_star1, sy_star1):
        ax1.annotate(r'$k*_1:$' + str(np.round(k_star1,2)) + "\n" + r'$sy*_1:$' + str(np.round(sy_star1,2)), (k_star1, sy_star1), textcoords="offset points", xytext=(5,-25))
    
    
    ############################## ax2 #####################################
    
    # ax2 for growth rate version
    ax2.title.set_text('Growth-Rate version')
    
    # Get and plot on ax2 s*y/k data
    investment_syk0 = s_y_div_k(x_vals, A0, alpha0, s0)
    ax2.plot(x_vals, investment_syk0, label=r'$sy/k_0$', color="red")
    
    # If (s0, A0, alpha0) and (s1, A1, alpha1) differ, draw a second sy line
    if (s0, A0, alpha0) != (s1, A1, alpha1):
        investment_syk1 = s_y_div_k(x_vals, A1, alpha1, s1)
        ax2.plot(x_vals, investment_syk1, label=r'$s(y/k)_1$', color="red", linestyle="dashed")
    
    # Draw n+delta
    ax2.plot([0, x_vals[len(x_vals)-1]], [n0+delta0]*2, color="blue", label=r'$(n+\delta)_0$' )
    
    # If (n0, delta0) and (n1, delta1) differ, draw a second sy line
    if (n0, delta0) != (n1, delta1):
        ax2.plot([0, x_vals[len(x_vals)-1]], [n1+delta1]*2, color="blue", label=r'$(n+\delta)_1$', linestyle="dashed" )
        
    # Set ylim
    max_n_plus_delta = max((n0+delta0), n1+delta1)
    ax2.set_ylim(0, 2*max_n_plus_delta)
    
    ############################## ax3 #####################################
    
    # Contributions to Growth
    
    # Contribution from investment 
    investment_syk0 = s_y_div_k(x_vals, A0, alpha0, s0)
    
    # Contribution from depreciation
    dep0 = [-delta0]*(len(x_vals))
    
    # Contribution from population growth
    pop0 = [-n0]*(len(x_vals))
    
    ax3.stackplot(x_vals, investment_syk0, dep0, pop0, labels=["investment", "depreciation", "pop growth"])
    
    ax3.set_ylim(-0.3,1)
    
    ax3.spines['bottom'].set_position('zero')

    
    
    
    
    for ax in (ax1, ax2, ax3): # Do this to keep axis labels which are lost with sharex = True
        ax.set_xlim(0, x_range_multiplier*max_kstar)
        ax.legend()    
    print(f'k*: {k_star0} \n k*: {y_star0}')
    
draw_btn.on_click(draw)
    
def investment_pc(k, A, alpha, s):
    """
    (np.array, num, num) -> np.array
    Returns np array of investment per person values from input np array of capital per person array, k
    """
    return s*A*k**alpha
    
def k_n_plus_delta(k,n,delta):
    """
    (np.array, num, num) -> np.array
    Returns np array of n + delta =values from input np array of capital per person array, k
    """
    return k*(n+delta)
    
def steady_state_vars(A,a,n,delta,s):
    """
    (num,num,num,num) -> (num, num)
    Returns a tuple of steady-state output (y*) and steady-state capital (k*)
    """
    k_star = ((s*A)/(n+delta))**(1/(1-a))
    y_star = A*(k_star**a)
    sy_star = s*y_star
    return (y_star,k_star,sy_star)


def s_y_div_k(k, A, alpha, s):
    """
    (np.array, num, num, num) -> np.array
    Returns s*(y/k) (= sA/k^(1-a)) for input of k
    """
    if min(k) == 0:
        k[0] = k[1]
    output = s*A/np.power(k,(1-alpha))
    return output
    

